# 2019년 10월 이커머스 데이터 EDA

In [22]:
# pip install plotly==5.8.0

In [1]:
import pandas as pd
import datetime as dt
from dateutil.parser import parse

In [2]:
df = pd.read_csv("D:/cp2_dataset/2019-Oct.csv")

In [3]:
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
...,...,...,...,...,...,...,...,...,...
42448759,2019-10-31 23:59:58 UTC,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193
42448760,2019-10-31 23:59:58 UTC,view,10800172,2053013554994348409,NaN,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8
42448761,2019-10-31 23:59:58 UTC,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd
42448762,2019-10-31 23:59:59 UTC,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204


## event_time 컬럼 나누기

In [4]:
utc_time = pd.to_datetime("2019-10-31 23:59:59 UTC")
utc_time

Timestamp('2019-10-31 23:59:59+0000', tz='UTC')

In [5]:
utc_time.tz_convert("Asia/Dubai")

Timestamp('2019-11-01 03:59:59+0400', tz='Asia/Dubai')

In [6]:
# event_time 컬럼 타입 str -> datetime64로 변경
df["event_time"] = pd.to_datetime(df["event_time"])

In [7]:
df["event_time"]

0          2019-10-01 00:00:00+00:00
1          2019-10-01 00:00:00+00:00
2          2019-10-01 00:00:01+00:00
3          2019-10-01 00:00:01+00:00
4          2019-10-01 00:00:04+00:00
                      ...           
42448759   2019-10-31 23:59:58+00:00
42448760   2019-10-31 23:59:58+00:00
42448761   2019-10-31 23:59:58+00:00
42448762   2019-10-31 23:59:59+00:00
42448763   2019-10-31 23:59:59+00:00
Name: event_time, Length: 42448764, dtype: datetime64[ns, UTC]

In [8]:
# event_time 열 timezone UTC -> Asia/Dubai로 변경 (GMT +4)
df["event_time"] = df["event_time"].apply(lambda x : x.tz_convert("Asia/Dubai"))

In [9]:
df["event_time"]

0          2019-10-01 04:00:00+04:00
1          2019-10-01 04:00:00+04:00
2          2019-10-01 04:00:01+04:00
3          2019-10-01 04:00:01+04:00
4          2019-10-01 04:00:04+04:00
                      ...           
42448759   2019-11-01 03:59:58+04:00
42448760   2019-11-01 03:59:58+04:00
42448761   2019-11-01 03:59:58+04:00
42448762   2019-11-01 03:59:59+04:00
42448763   2019-11-01 03:59:59+04:00
Name: event_time, Length: 42448764, dtype: datetime64[ns, Asia/Dubai]

In [10]:
# event_time 월, 일, 요일, 시 빼내기
df["month"] = df["event_time"].dt.month
df["day"] = df["event_time"].dt.day
df["day_name"] = df["event_time"].apply(lambda x : x.day_name())
df["hour"] = df["event_time"].dt.hour

In [11]:
df.columns

Index(['event_time', 'event_type', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session', 'month',
       'day', 'day_name', 'hour'],
      dtype='object')

In [12]:
column_order = ['event_time', 'month', 'day', 'day_name', 'hour', 
                'event_type', 'product_id', 'category_id', 'category_code', 
                'brand', 'price', 'user_id', 'user_session']
df_mod = df[column_order]

In [13]:
df_mod.head()

,event_time,month,day,day_name,hour,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 04:00:04+04:00,10,1,Tuesday,4,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [28]:
df["event_type"].value_counts(normalize=True)

view        0.960673
cart        0.021827
purchase    0.017500
Name: event_type, dtype: float64

In [27]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df_mod["day_name"])
)
fig.show()

MemoryError: 

In [4]:
1+5

6

In [15]:
# event_time 월만 빼내기
df["event_time"].dt.month

0           10
1           10
2           10
3           10
4           10
            ..
42448759    11
42448760    11
42448761    11
42448762    11
42448763    11
Name: event_time, Length: 42448764, dtype: int64

In [16]:
# event_time 일만 빼내기
df["event_time"].dt.day

0           1
1           1
2           1
3           1
4           1
           ..
42448759    1
42448760    1
42448761    1
42448762    1
42448763    1
Name: event_time, Length: 42448764, dtype: int64

In [17]:
# event_time 시간만 빼내기
df["event_time"].dt.hour

0           4
1           4
2           4
3           4
4           4
           ..
42448759    3
42448760    3
42448761    3
42448762    3
42448763    3
Name: event_time, Length: 42448764, dtype: int64

In [18]:
# event_time 분만 빼내기
df["event_time"].dt.minute

0            0
1            0
2            0
3            0
4            0
            ..
42448759    59
42448760    59
42448761    59
42448762    59
42448763    59
Name: event_time, Length: 42448764, dtype: int64

In [19]:
# event_time 요일 이름만 빼내기
df["event_time"].apply(lambda x : x.day_name())

0           Tuesday
1           Tuesday
2           Tuesday
3           Tuesday
4           Tuesday
             ...   
42448759     Friday
42448760     Friday
42448761     Friday
42448762     Friday
42448763     Friday
Name: event_time, Length: 42448764, dtype: object

In [6]:
1+10

11

In [20]:
df["category_code"].str.split(".").str[0]

MemoryError: 